Este programa en Python tiene como objetivo filtrar y extraer información de usuarios que se han conectado a un punto de acceso (AP) en un rango de fechas específico. Está compuesto por dos módulos: csv_tool.py y app_cli.py.

- csv_tool: se encarga de extraer la informacion y aplicar expresiones regulares
- app_cli: se encarga de filtar la informacion con la MACAddress del AP elegido y con las fechas provistas por el usuario

csv_tool

Primero lo que haces es definir las expresiones regulares que vamos a usar.

Las expresiones regulares (regex) son patrones de búsqueda utilizados para encontrar coincidencias dentro de texto. En este programa, se utilizan varias expresiones regulares para validar y filtrar datos en el archivo CSV.


In [ ]:
import re

MAC_AP_RE = re.compile(r"([0-9A-Fa-f]{2}-){5}[0-9A-Fa-f]{2}:HCDD$")
MAC_CLIENT_RE = re.compile(r"([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$")
IP_NAS_RE = re.compile(r"((?:192\.168\.247\.[0-9]{2})|(?:192\.168\.1\.20))$")
ID_RE = re.compile(r"\d{6,7}$")
ID_SESION_RE = re.compile(r"[A-F0-9]{8}-[A-F0-9]{8}$")
ID_CONEXION_UNICO_RE = re.compile(r"[a-f0-9]{16}$")
USER_RE = re.compile(r"[a-zA-Z.-]{3,25}$")
DATE_RE = re.compile(r"(20(1[5-9]|2[0-5])[-/](0[1-9]|1[0-2])[-/]([0-2]\d|3[0-1])$)")
HOUR_RE = re.compile(r"([01]\d|2[0-3]):([0-5]\d):([0-5]\d)$")


Luego tiene 2 funciones:

create_pandas:
Se encarga de cargar los datos desde un archivo CSV en un DataFrame de Pandas. El proceso se realiza en tres pasos:

1-Se importa la librería Pandas con import pandas as pd.

2-Se utiliza "pd.read_csv(csv_path, low_memory=False)" para cargar los datos del archivo CSV especificado en csv_path en un DataFrame de Pandas llamado data. El parámetro "low_memory=False" se utiliza para evitar problemas de manejo de memoria en archivos CSV grandes.

3-La función devuelve el DataFrame data que contiene los datos del archivo CSV.

In [ ]:
import pandas as pd

def create_pandas(csv_path: str) -> pd.DataFrame:
    data = pd.read_csv(csv_path, low_memory=False)
    return data

Y la funcion apply_regex:

La función apply_regex se utiliza para aplicar expresiones regulares a un DataFrame de Pandas y filtrar los datos que coinciden con los patrones definidos. El proceso se realiza en los siguientes pasos:

1-Se define una variable filtro que es una serie booleana que representa qué filas cumplen con las condiciones especificadas por las expresiones regulares. Cada expresión regular se aplica a una columna específica del DataFrame.

2-Luego, se aplica este filtro al DataFrame utilizando data[filtro], lo que resulta en un nuevo DataFrame que contiene solo las filas que coinciden con los patrones de las expresiones regulares.

3-Finalmente, se devuelve este nuevo DataFrame con los datos filtrados.

In [ ]:
def apply_regex(data: pd.DataFrame) -> pd.DataFrame:
    filtro = (
        data["MAC_AP"].apply(lambda x: bool(re.fullmatch(MAC_AP_RE, str(x))))
        & data["MAC_Cliente"].apply(lambda x: bool(re.fullmatch(MAC_CLIENT_RE, str(x))))
        & data["IP_NAS_AP"].apply(lambda x: bool(re.fullmatch(IP_NAS_RE, str(x))))
        & data["ID"].apply(lambda x: bool(re.fullmatch(ID_RE, str(x))))
        & data["ID_Sesion"].apply(lambda x: bool(re.fullmatch(ID_SESION_RE, str(x))))
        & data["ID_Conexión_unico"].apply(lambda x: bool(re.fullmatch(ID_CONEXION_UNICO_RE, str(x))))
        & data["Inicio_de_Conexión_Dia"].apply(lambda x: bool(re.fullmatch(DATE_RE, str(x))))
        & data["FIN_de_Conexión_Dia"].apply(lambda x: bool(re.fullmatch(DATE_RE, str(x))))
        & data["Inicio_de_Conexión_Hora"].apply(lambda x: bool(re.fullmatch(HOUR_RE, str(x))))
        & data["FIN_de_Conexión_Hora"].apply(lambda x: bool(re.fullmatch(HOUR_RE, str(x))))
        & data["Usuario"].apply(lambda x: bool(re.fullmatch(USER_RE, str(x))))
    )
    data = data[filtro]
    return data

Módulo app_cli.py:

Este módulo se encarga de la interacción con el usuario para realizar consultas en el archivo CSV filtrado.

Lectura y procesamiento del CSV: 

Este bloque lee el archivo CSV y aplica las expresiones regulares definidas en csv_tool.py para limpiar y validar los datos.

In [ ]:
import typer
from yaspin import yaspin
from csv_tools import create_pandas, apply_regex

def menu(csv_path: str = typer.Option(..., prompt="Ingrese la ruta del archivo csv")):
    INICIO_CONEXION_DIA = "Inicio_de_Conexión_Dia"
    FIN_CONEXION_DIA = "FIN_de_Conexión_Dia"

    with yaspin(text="Leyendo csv", color="yellow"):
        data = create_pandas(csv_path)
    with yaspin(text="Aplicando expresiones regulares", color="yellow"):
        data = apply_regex(data)
    ap_list = data["MAC_AP"].unique().tolist()


Selección de Access Point (AP) y Fechas:

Dentro de un bucle while True, la función permite al usuario seleccionar un Access Point (AP) de una lista de AP únicos presentes en el archivo CSV.
Luego, solicita al usuario que ingrese fechas de inicio y fin en formato "YYYY-MM-DD".

In [ ]:
import inquirer

ap_list = data["MAC_AP"].unique().tolist()
while True:
        ap = inquirer.prompt(
            [inquirer.List("ACCESS POINT", message="Seleccione un AP", choices=ap_list)]
        )

        fechas = inquirer.prompt(
            [
                inquirer.Text(
                    "FECHA DE INICIO",
                    message="Ingrese la fecha de inicio (YYYY-MM-DD)",
                    validate=lambda _, x: DATE_RE.fullmatch(x) is not None,
                ),
                inquirer.Text(
                    "FECHA DE FIN",
                    message="Ingrese la fecha de fin (YYYY-MM-DD)",
                    validate=lambda _, x: DATE_RE.fullmatch(x) is not None,
                ),
            ]
        )

        ap, fecha_i, fecha_f = (
            ap["ACCESS POINT"],
            pd.to_datetime(fechas["FECHA DE INICIO"]),
            pd.to_datetime(fechas["FECHA DE FIN"]),
        )

        data["Inicio_de_Conexión_Dia"] = pd.to_datetime(data["Inicio_de_Conexión_Dia"])
        data["FIN_de_Conexión_Dia"] = pd.to_datetime(data["FIN_de_Conexión_Dia"]) 

Filtrado de datos:

Después de obtener la selección del usuario, se aplica un filtro a los datos del archivo CSV.
El filtro busca registros que coincidan con el Access Point seleccionado y que estén dentro del rango de fechas especificado.
También se tiene en cuenta si las fechas de inicio o fin de conexión están dentro del rango especificado.
Los registros que cumplen con estas condiciones se almacenan en la variable data.

In [ ]:
ap, fecha_i, fecha_f = (
    ap["ACCESS POINT"],
    pd.to_datetime(fechas["FECHA DE INICIO"]),
    pd.to_datetime(fechas["FECHA DE FIN"]),
)

data[INICIO_CONEXION_DIA] = pd.to_datetime(data[INICIO_CONEXION_DIA])
data[FIN_CONEXION_DIA] = pd.to_datetime(data[FIN_CONEXION_DIA])

filtro = (
    (data["MAC_AP"] == ap) &
    (
        (data[INICIO_CONEXION_DIA].between(fecha_i, fecha_f)) |
        (data[FIN_CONEXION_DIA].between(fecha_i, fecha_f)) |
        (data[INICIO_CONEXION_DIA] <= fecha_i) & (data[FIN_CONEXION_DIA] >= fecha_f)
    )
)

Extracción de Usuarios Únicos y Presentación de Resultados:

La función extrae los nombres de usuario únicos de los registros filtrados en data.
Se genera una consulta que describe la selección del usuario (AP, fechas) y se muestra en la consola.
Los nombres de usuario únicos se presentan en forma de tabla en la consola utilizando la función tabulate.

In [ ]:
import tabulate

data = data.loc[filtro]
users = data["Usuario"].unique()

consulta = f'Usuarios conectados al AP {ap} entre {fecha_i} y {fecha_f}'
print("")
print(
    tabulate(
        [[user] for user in users],
        headers=[
            consulta
        ],
        tablefmt="grid",
        stralign="center",
    )
)


Opciones Adicionales:

Se le pregunta al usuario si desea guardar los datos filtrados en un nuevo archivo CSV, en caso de afirmar utiliza la funcion "write_to_csv.
También se le pregunta si desea realizar otra consulta.

In [ ]:
if typer.confirm("¿Desea guardar los datos en un archivo csv?"):
    write_to_csv(data, consulta)

if not typer.confirm("¿Desea realizar otra consulta?"):
    print("Gracias por usar nuestro programa")
    break

def write_to_csv(data: pd.DataFrame, consulta: str):
    file_name = f'output_{consulta}.csv'
    data.to_csv(file_name, index=False)